In [1]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [2]:
args = Namespace(
    raw_data="../data/ag_news/news.csv",
    train_prop=0.7,
    val_prop=0.15,
    test_prop=0.15,
    processed_data="../data/ag_news/ag_news_processed_data.csv",
    seed=1337
)

In [3]:
# Read raw data
news = pd.read_csv(args.raw_data, header=0)

In [4]:
news.head()

,category,title
0,Business,Wall St. Bears Claw Back Into the Black (Reuters)
1,Business,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Business,Oil and Economy Cloud Stocks' Outlook (Reuters)
3,Business,Iraq Halts Oil Exports from Main Southern Pipe...
4,Business,"Oil prices soar to all-time record, posing new..."


In [5]:
# Unique classes
set(news.category)

{'Business', 'Sci/Tech', 'Sports', 'World'}

In [6]:
# Splitting train by category
# Create dict
by_category = collections.defaultdict(list)
for _, row in news.iterrows():
    by_category[row.category].append(row.to_dict())

In [7]:
# Create split data
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_category.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_prop*n)
    n_val = int(args.val_prop*n)
    n_test = int(args.test_prop*n)
    
    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    # Add to final list
    final_list.extend(item_list)

In [8]:
# Write split data to file
final_news = pd.DataFrame(final_list)

In [9]:
final_news.split.value_counts()

train    84000
test     18000
val      18000
Name: split, dtype: int64

In [10]:
# Preprocess the reviews
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
    
final_news.title = final_news.title.apply(preprocess_text)

In [11]:
final_news.head()

,category,title,split
0,Business,"jobs , tax cuts key issues for bush",train
1,Business,jarden buying mr . coffee s maker,train
2,Business,retail sales show festive fervour,train
3,Business,intervoice s customers come calling,train
4,Business,boeing expects air force contract,train


In [12]:
# Write munged data to CSV
final_news.to_csv(args.processed_data, index=False)